## Catching a Falling Initial

In this notebook, we will learn how to catch an object that is falling with a Kuka iiwa-7. This task is difficult because of the timing components. You will be responsible for calculating the rate at which an object is falling using several Intel Realsense cameras and then calculating the expected position of the object at a certain point in time, where you will catch the object.


**Learning Objectives:**
1. Implement a system that uses a known location of a falling item to catch a falling initial
2. Implement control (probably force control) to catch a falling object without it hitting the ground.

This assignment took a lot of inspiration from the Geometric Pick and Place and a lot of the structure may reflect this

## Setup and imports
Let us first get our imports out of the way:

In [1]:
import os
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import trimesh
from pydrake.all import (
    AddFrameTriadIllustration,
    BasicVector,
    Concatenate,
    Context,
    Diagram,
    DiagramBuilder,
    Integrator,
    JacobianWrtVariable,
    LeafSystem,
    MultibodyPlant,
    PiecewisePolynomial,
    PiecewisePose,
    RigidTransform,
    RobotDiagram,
    RollPitchYaw,
    RotationMatrix,
    Simulator,
    StartMeshcat,
    Trajectory,
    TrajectorySource,
)

from manipulation import running_as_notebook
from manipulation.exercises.grader import Grader
from manipulation.icp import IterativeClosestPoint
from manipulation.letter_generation import create_sdf_asset_from_letter
from manipulation.meshcat_utils import AddMeshcatTriad
from manipulation.station import (
    AddPointClouds,
    LoadScenario,
    MakeHardwareStation,
    RobotDiagram,
)
from manipulation.utils import RenderDiagram

/usr/local/lib/python3.12/dist-packages/matplotlib/projections/__init__.py:63: UserWarning: Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.
  warnings.warn("Unable to import Axes3D. This may be due to multiple versions of "


### Meshcat Visualization

As always, let's start Meshcat for our 3D visualization!

In [2]:
# Start meshcat for visualization
meshcat = StartMeshcat()
print("Click the link above to open Meshcat in your browser!")

INFO:drake:Meshcat listening for connections at http://localhost:7000


Click the link above to open Meshcat in your browser!


In [3]:
# A note: as an additional argument add `include_normals=True` to the `create_sdf_asset_from_letter` function.
# We also recommend you use a slightly smaller `letter_height_meters=0.15` and a rather high friction coefficient
# (we use mu_static = 1.17, mu_dynamic = 1.0 - roughly equal to the friction of rubber on granite) to make the
# letters easier to pick up. We'll study more complicated grasping strategies in future notebooks, but this is not
# the priority of this chapter so we'll cheat.

from pathlib import Path
from manipulation.letter_generation import create_sdf_asset_from_letter

output_dir = Path("assets")
output_dir.mkdir(parents=True, exist_ok=True)

your_initial = "I"

create_sdf_asset_from_letter(
    your_initial,             # just the letter as the first positional arg
    output_dir=output_dir,    # keep this as a keyword
    include_normals=True,     # might be include_normal in your version
    letter_height_meters=0.25,
    mu_static=1.17,
    mu_dynamic=1.0,
)

PosixPath('assets/I.sdf')

In [4]:
# Creating a Table SDF that the letter will fall onto (potentially if I have time, I'll setit up to fall somewhere randomly in this space)
size_x = 2
size_y = 2
thickness = 0.05
mass = 10.0
mu_static = 1.0
mu_dynamic = 0.8
dissipation = 1.0

Ixx = (1.0/12.0) * mass * (size_y**2 + thickness**2)
Iyy = (1.0/12.0) * mass * (size_x**2 + thickness**2)
Izz = (1.0/12.0) * mass * (size_x**2 + size_y**2)

table_sdf = f"""<?xml version="1.0"?>
<sdf version="1.7" xmlns:drake="http://drake.mit.edu">
  <model name="table">
    <link name="table_top">
      <inertial>
        <mass>{mass}</mass>
        <inertia>
          <ixx>{Ixx}</ixx><iyy>{Iyy}</iyy><izz>{Izz}</izz>
          <ixy>0</ixy><ixz>0</ixz><iyz>0</iyz>
        </inertia>
      </inertial>

      <visual name="visual">
        <geometry><box><size>{size_x} {size_y} {thickness}</size></box></geometry>
        <material>
          <ambient>0.6 0.6 0.6 1</ambient>
          <diffuse>0.7 0.7 0.7 1</diffuse>
        </material>
      </visual>

      <collision name="collision">
        <geometry><box><size>{size_x} {size_y} {thickness}</size></box></geometry>
        <drake:proximity_properties>
          <drake:hunt_crossley_dissipation>{dissipation}</drake:hunt_crossley_dissipation>
          <drake:mu_static>{mu_static}</drake:mu_static>
          <drake:mu_dynamic>{mu_dynamic}</drake:mu_dynamic>
        </drake:proximity_properties>
      </collision>
    </link>
  </model>
</sdf>

"""

assets_dir = Path("assets")
assets_dir.mkdir(parents=True, exist_ok=True)
(assets_dir / "table.sdf").write_text(table_sdf)
print("Wrote", (assets_dir / "table.sdf").resolve())

Wrote /workspaces/Projects/robotic_manipulation/00-Pset-9/assets/table.sdf


In [5]:
# Add the directives for the bimanual IIWA arms, table, and initials

def generate_bimanual_IIWA14_with_assets_directives_file() -> (
    tuple[Diagram, RobotDiagram]
):
    table_sdf = f"{Path.cwd()}/assets/table.sdf"
    letter_sdf = f"{Path.cwd()}/assets/{your_initial}.sdf"

    directives_yaml = f"""directives:
- add_model:
    name: table
    file: file://{table_sdf}
- add_weld:
    parent: world
    child: table::table_top
    X_PC:
        translation: [0.0, 0.0, -.025]
        rotation: !Rpy {{ deg: [0, 0, -90] }}
- add_model:
    name: iiwa
    file: package://drake_models/iiwa_description/sdf/iiwa7_no_collision.sdf
    default_joint_positions:
        iiwa_joint_1: [0.0]
        iiwa_joint_2: [0.5]
        iiwa_joint_3: [0.0]
        iiwa_joint_4: [-1.8]
        iiwa_joint_5: [0.0]
        iiwa_joint_6: [1.2]
        iiwa_joint_7: [1.0]
- add_weld:
    parent: table::table_top
    child: iiwa::iiwa_link_0
    X_PC:
        translation: [.5, 0, .025]
        rotation: !Rpy {{ deg: [0, 0, -90] }}
- add_model:
    name: wsg
    file: package://manipulation/hydro/schunk_wsg_50_with_tip.sdf
- add_weld:
    parent: iiwa::iiwa_link_7
    child: wsg::body
    X_PC:
        translation: [0, 0, 0.09]
        rotation: !Rpy {{ deg: [90, 0, 90]}}





- add_model:
    name: cheez_it
    file: package://manipulation/hydro/003_cracker_box.sdf


"""
    os.makedirs("directives", exist_ok=True)

    with open(
        "directives/bimanual_IIWA14_with_table_and_initials_and_assets.dmd.yaml", "w"
    ) as f:
        f.write(directives_yaml)




        # iiwa_joint_1: [-1.57]
        # iiwa_joint_2: [0.1]
        # iiwa_joint_3: [0]
        # iiwa_joint_4: [-1.2]
        # iiwa_joint_5: [0]
        # iiwa_joint_6: [1.6]
        # iiwa_joint_7: [0]

        # iiwa_joint_1: [0]
        # iiwa_joint_2: [0]
        # iiwa_joint_3: [0]
        # iiwa_joint_4: [0]
        # iiwa_joint_5: [0]
        # iiwa_joint_6: [0]
        # iiwa_joint_7: [0]

        # iiwa_joint_1: [0.0]
        # iiwa_joint_2: [0.5]
        # iiwa_joint_3: [0.0]
        # iiwa_joint_4: [-1.8]
        # iiwa_joint_5: [0.0]
        # iiwa_joint_6: [1.2]
        # iiwa_joint_7: [0.0]


# - add_model:
#     name: {your_initial}_letter
#     file: file://{letter_sdf}


generate_bimanual_IIWA14_with_assets_directives_file()

In [6]:
def create_bimanual_IIWA14_with_assets_scenario() -> None:
    # Build absolute file:// URIs for the two directives files
    dmd1 = (Path.cwd() / "directives" / "bimanual_IIWA14_with_table_and_initials_and_assets.dmd.yaml").resolve().as_posix()
    # dmd2 = (Path.cwd() / "directives" / "camera_directives.dmd.yaml").resolve().as_posix()

    # Part 1: directives (use file:// URIs)
    scenario_yaml = f"""# Station scenario
directives:
  - add_directives:
      file: file://{dmd1}
"""

    # Part 2: drivers
    scenario_yaml += """
model_drivers:
  iiwa: !IiwaDriver
    control_mode: position_only
    hand_model_name: wsg
  wsg: !SchunkWsgDriver {}
"""

    os.makedirs("scenarios", exist_ok=True)

    with open(
        "scenarios/bimanual_IIWA14_with_table_and_initials_and_assets.scenario.yaml",
        "w",
    ) as f:
        f.write(scenario_yaml)


create_bimanual_IIWA14_with_assets_scenario()

In [7]:
def create_bimanual_IIWA14_with_table_and_initials_and_assets() -> (
    tuple[DiagramBuilder, RobotDiagram]
):
    # TODO: Load the scenario created above into a Scenario object
    scenario = LoadScenario(
        filename="scenarios/bimanual_IIWA14_with_table_and_initials_and_assets.scenario.yaml"
    )

    # TODO: Create HardwareStation with the scenario and meshcat
    station = MakeHardwareStation(scenario, meshcat=meshcat)

    # TODO: Make a DiagramBuilder, add the station, and build the diagram
    builder = DiagramBuilder()
    builder.AddSystem(station)

    # TODO: Return the builder AND the station (notice that here we will need both)
    return builder, station


In [8]:
import numpy as np
from pydrake.all import RigidTransform, RotationMatrix, RollPitchYaw

def design_grasp_pose_for_catching(X_WO: RigidTransform) -> tuple[RigidTransform, RigidTransform]:
    """
    Super simple catching strategy:
    1. Get the object's predicted world position
    2. Place gripper some distance below it
    3. Keep gripper orientation simple - fingers horizontal, approaching from below

    Args:
        X_WO: Predicted pose of the object (letter) at catch time

    Returns:
        X_OG: Gripper pose relative to object (for reference)
        X_WG: Gripper pose in world frame (what we actually use)
    """
    # Get where the object will be
    p_WO = X_WO.translation()

    # Step 1: Design a simple gripper orientation
    # For WSG gripper:
    # - Z axis = approach direction (should point UP toward falling object)
    # - Y axis = finger open/close direction (should be HORIZONTAL)
    # - X axis = perpendicular to both

    # Simple approach: gripper "standing up" with fingers horizontal
    # This is just a 180° rotation about X axis from default orientation
    R_WG = RotationMatrix.MakeXRotation(np.pi)

    # Step 2: Place gripper below the object
    # Just offset downward in world Z
    catch_offset = 0.1  # 10cm below object center
    p_WG = p_WO.copy()
    gripping_point_offset_in_gripper_frame = np.array([0, 0.1, 0])
    gripping_point_offset_in_world = R_WG @ gripping_point_offset_in_gripper_frame
    p_WG = p_WG - gripping_point_offset_in_world  # Adjust so gripping point is at desired location


    p_WG[2] -= catch_offset  # Move down in world frame

    # Build the world-frame gripper pose
    X_WG = RigidTransform(R_WG, p_WG)

    # Also compute X_OG for completeness (though we don't really need it)
    X_WO_inv = X_WO.inverse()
    X_OG = X_WO_inv @ X_WG

    return X_OG, X_WG



def design_pregrasp_pose(X_WG: RigidTransform, lower_by: float = 0.15) -> RigidTransform:
    """
    Pre-grasp: just move the gripper further down and maybe slightly back.
    Keep the same orientation.

    Args:
        X_WG: Target catch pose
        lower_by: How much lower to position the pre-grasp (meters)
    """
    # Start from the catch pose
    p_WGpre = X_WG.translation().copy()
    R_WG = X_WG.rotation()

    # Move further down in world Z
    p_WGpre[2] -= lower_by

    # Optionally: move slightly back along gripper's -Z axis
    # (this moves "away" from the approach direction)
    z_axis_in_world = R_WG.matrix()[:, 2]
    p_WGpre -= 0.05 * z_axis_in_world  # 5cm back

    return RigidTransform(R_WG, p_WGpre)


def verify_grasp_geometry(X_WO_catch: RigidTransform, X_WG_catch: RigidTransform):
    """
    Print debug info about the grasp.
    """
    p_WO = X_WO_catch.translation()
    p_WG = X_WG_catch.translation()

    print("\n" + "="*50)
    print("GRASP GEOMETRY CHECK")
    print("="*50)
    print(f"Object position (world):  x={p_WO[0]:6.3f}, y={p_WO[1]:6.3f}, z={p_WO[2]:6.3f}")
    print(f"Gripper position (world): x={p_WG[0]:6.3f}, y={p_WG[1]:6.3f}, z={p_WG[2]:6.3f}")
    print(f"Vertical offset: {p_WG[2] - p_WO[2]:.3f}m")

    # Check gripper is below object
    if p_WG[2] < p_WO[2]:
        print("✓ Gripper is below object")
    else:
        print("✗ WARNING: Gripper is ABOVE object!")

    # Check gripper orientation
    R_WG = X_WG_catch.rotation()
    z_axis = R_WG.matrix()[:, 2]  # Approach direction
    y_axis = R_WG.matrix()[:, 1]  # Finger direction

    print(f"\nGripper orientation:")
    print(f"  Z-axis (approach): [{z_axis[0]:5.2f}, {z_axis[1]:5.2f}, {z_axis[2]:5.2f}]", end="")
    if z_axis[2] > 0.7:
        print(" ✓ (pointing UP)")
    else:
        print(" ✗ (should point UP!)")

    print(f"  Y-axis (fingers):  [{y_axis[0]:5.2f}, {y_axis[1]:5.2f}, {y_axis[2]:5.2f}]", end="")
    if abs(y_axis[2]) < 0.3:
        print(" ✓ (horizontal)")
    else:
        print(" ⚠ (should be horizontal)")

    print("="*50 + "\n")


def make_bot_trajectory(
    X_Gs: list[RigidTransform],
    finger_values: np.ndarray,
    sample_times: list[float]
) -> tuple:
    """
    Create trajectories for gripper pose and finger positions.

    Returns:
        robot_velocity_trajectory: Spatial velocity trajectory for controller
        traj_wsg_command: Finger position trajectory
    """
    from pydrake.all import PiecewisePose, PiecewisePolynomial

    robot_position_trajectory = PiecewisePose.MakeLinear(sample_times, X_Gs)
    robot_velocity_trajectory = robot_position_trajectory.MakeDerivative()
    traj_wsg_command = PiecewisePolynomial.FirstOrderHold(sample_times, finger_values)

    return robot_velocity_trajectory, traj_wsg_command

def solve_catch_time(z0, vz0, z_target, g=-9.81):
    """Solve quadratic equation for time to reach target height"""
    a = 0.5 * g
    b = vz0
    c = z0 - z_target

    discriminant = b**2 - 4*a*c
    if discriminant < 0:
        print("WARNING: Object won't reach target height!")
        return None

    t1 = (-b + np.sqrt(discriminant)) / (2*a)
    t2 = (-b - np.sqrt(discriminant)) / (2*a)

    # Take the positive time
    valid_times = [t for t in [t1, t2] if t > 0]
    return min(valid_times) if valid_times else None

In [9]:
# TODO: copy over the pseudoinverse controller from the previous pset below.
# OPTIONAL: if you want to you can try designing a more complicated controller for the pick and place.


class PseudoInverseController(LeafSystem):
    def __init__(self, plant: MultibodyPlant):
        LeafSystem.__init__(self)
        self._plant = plant
        self._plant_context = plant.CreateDefaultContext()
        self._iiwa = plant.GetModelInstanceByName("iiwa")
        self._G = plant.GetBodyByName("body").body_frame()
        self._W = plant.world_frame()

        self.V_G_port = self.DeclareVectorInputPort("V_WG", 6)
        self.q_port = self.DeclareVectorInputPort("iiwa.position", 7)
        self.DeclareVectorOutputPort("iiwa.velocity", 7, self.CalcOutput)
        self.iiwa_start = plant.GetJointByName("iiwa_joint_1").velocity_start()
        self.iiwa_end = plant.GetJointByName("iiwa_joint_7").velocity_start()

    def CalcOutput(self, context: Context, output: BasicVector):
        """
        fill in our code below.
        """
        # evaluate the V_G_port and q_port on the current context to get those values.
        V_G = V_G = self.V_G_port.Eval(context)
        q = self.q_port.Eval(context)

        # update the positions of the internal _plant_context according to `q`.
        # HINT: you can write to a plant context by calling `self._plant.SetPositions`

        self._plant.SetPositions(self._plant_context, self._iiwa, q)

        # Compute the gripper jacobian
        # HINT: the jacobian is 6 x N, with N being the number of DOFs.
        # We only want the 6 x 7 submatrix corresponding to the IIWA
        J_full = self._plant.CalcJacobianSpatialVelocity(
            self._plant_context,
            JacobianWrtVariable.kV,
            self._G, np.zeros(3),   # p_BoQ_B = [0,0,0] at frame G origin
            self._W,                # measured in world (Ao = W)
            self._W                 # expressed in world (E = W)
        )
        J = J_full[:, self.iiwa_start : self.iiwa_end + 1]

        # compute `v` by mapping the gripper velocity (from the V_G_port) to the joint space
        lam2 = 1e-6
        JJt = J @ J.T
        J_pinv = J.T @ np.linalg.inv(JJt + lam2 * np.eye(6))
        v = J_pinv @ V_G  # 7x1 joint velocity command

        output.SetFromVector(v)


def get_model_body(plant: MultibodyPlant):
    ""

    # Get the letter body
    # letter_model = plant.GetModelInstanceByName(f"{your_initial}_letter")
    # letter_body = plant.GetBodyByName(f"{your_initial}_body_link", letter_model)


    # - add_model:
    #     name: cheez_it
    #     file: package://manipulation/hydro/003_cracker_box.sdf

    letter_model = plant.GetModelInstanceByName(f"cheez_it")
    letter_body = plant.GetBodyByName("base_link_cracker", letter_model)

    return letter_model, letter_body


def visualize_everything(station, plant):
    # visualize axes (useful for debugging)
    scenegraph = station.GetSubsystemByName("scene_graph")
    AddFrameTriadIllustration(
        scene_graph=scenegraph,
        body=plant.GetBodyByName("iiwa_link_0"),
        length=4,
    )
    # AddFrameTriadIllustration(
    #     scene_graph=scenegraph,
    #     body=plant.GetBodyByName(f"{your_initial}_body_link"),
    #     length=0.1,
    # )
    AddFrameTriadIllustration(
        scene_graph=scenegraph,
        body=plant.GetBodyByName("base_link_cracker"),
        length=0.1,
    )
    AddFrameTriadIllustration(
        scene_graph=scenegraph, body=plant.GetBodyByName("body"), length=0.1
    )

def solve_catch_time(z0, vz0, z_target, g=-9.81):
    """Solve quadratic equation for time to reach target height"""
    a = 0.5 * g
    b = vz0
    c = z0 - z_target

    discriminant = b**2 - 4*a*c
    if discriminant < 0:
        print("WARNING: Object won't reach target height!")
        return None

    t1 = (-b + np.sqrt(discriminant)) / (2*a)
    t2 = (-b - np.sqrt(discriminant)) / (2*a)

    # Take the positive time
    valid_times = [t for t in [t1, t2] if t > 0]
    return min(valid_times) if valid_times else None


In [ ]:
builder, station = create_bimanual_IIWA14_with_table_and_initials_and_assets()
plant = station.GetSubsystemByName("plant")

# Create a temporary minimal diagram just to get simulator context
temp_builder = DiagramBuilder()
temp_builder.AddSystem(station)
temp_diagram = temp_builder.Build()

# Create simulator to get the actual starting state
temp_simulator = Simulator(temp_diagram)
temp_context = temp_simulator.get_mutable_context()
temp_plant_context = plant.GetMyContextFromRoot(temp_context)

# Get X_WGinitial from simulator context (this is the actual starting pose)
X_WGinitial = plant.EvalBodyPoseInWorld(temp_plant_context, plant.GetBodyByName("body"))
print("X_WGinitial from simulator context:", X_WGinitial)

# Set letter position in simulator context
letter_model, letter_body = get_model_body(plant)
p0_letter = np.array([0.0, 0.0, 21.5])
R_WL = RollPitchYaw(np.pi/2, np.pi, np.pi/2)
plant.SetFreeBodyPose(
    temp_plant_context,
    letter_body,
    RigidTransform(R_WL, p0_letter),
)

# Now compute trajectory with correct X_WGinitial
v0_letter = np.array([0.0, 0.0, 0.0])
catch_height = 1.5
t_catch = solve_catch_time(p0_letter[2], v0_letter[2], catch_height)
if t_catch is None:
    print("ERROR: Can't calculate catch time!")
    t_catch = 1.0

g = -9.81
p_catch = p0_letter + v0_letter * t_catch + 0.5 * np.array([0, 0, g]) * t_catch**2
R_catch = RotationMatrix.MakeZRotation(np.pi/2)
X_WO_catch = RigidTransform(R_catch, p_catch)

X_OG, X_WGpick = design_grasp_pose_for_catching(X_WO_catch)
X_WGprepick = design_pregrasp_pose(X_WGpick)
verify_grasp_geometry(X_WO_catch, X_WGpick)

opened = 0.107
closed = 0.0
min_spacing = 0.15

if t_catch < 2.0:
    keyframes = [
        ("X_WGinitial", X_WGinitial, opened, 0.0),
        ("X_WGprepick", X_WGprepick, opened, max(0.5, t_catch * 0.3)),
        ("X_WGpick", X_WGpick, opened, max(0.8, t_catch * 0.7)),
        ("X_WGpick", X_WGpick, closed, t_catch + 0.2),
        ("X_WGpick", X_WGpick, closed, t_catch + 1.5),
    ]
else:
    keyframes = [
        ("X_WGinitial", X_WGinitial, opened, 0.0),
        ("X_WGprepick", X_WGprepick, opened, t_catch - 1.5),
        ("X_WGpick", X_WGpick, opened, t_catch - 0.3),
        ("X_WGpick", X_WGpick, closed, t_catch + 0.2),
        ("X_WGpick", X_WGpick, closed, t_catch + 1.5),
    ]

sample_times = [keyframe[3] for keyframe in keyframes]
for i in range(1, len(sample_times)):
    if sample_times[i] - sample_times[i-1] < min_spacing:
        sample_times[i] = sample_times[i-1] + min_spacing

gripper_poses = [keyframe[1] for keyframe in keyframes]
finger_states = np.asarray([keyframe[2] for keyframe in keyframes]).reshape(1, -1)
traj_V_G, traj_wsg_command = make_bot_trajectory(gripper_poses, finger_states, sample_times)

INFO:drake:PackageMap: Downloading https://github.com/RobotLocomotion/models/archive/7b92aacbe021861ec9bbbb82d8ab9a19ded970ff.tar.gz


X_WGinitial from simulator context: RigidTransform(
  R=RotationMatrix([
    [-0.7880011308845268, 0.3507832276896201, 0.505969707488857],
    [-0.5403023058681402, -3.052086405318471e-16, -0.8414709848078967],
    [-0.295173908058077, -0.9364566872907975, 0.18952898678057062],
  ]),
  p=[-0.42963119461589844, -0.5000000000000003, 0.2652510977908333],
)

GRASP GEOMETRY CHECK
Object position (world):  x= 0.000, y= 0.000, z= 1.500
Gripper position (world): x= 0.000, y= 0.100, z= 1.400
Vertical offset: -0.100m
✓ Gripper is below object

Gripper orientation:
  Z-axis (approach): [ 0.00, -0.00, -1.00] ✗ (should point UP!)
  Y-axis (fingers):  [ 0.00, -1.00,  0.00] ✓ (horizontal)



In [11]:
# Now build the full diagram with trajectory systems
# (Recreate builder since we used temp_builder above, or reuse the original)
builder, station = create_bimanual_IIWA14_with_table_and_initials_and_assets()
plant = station.GetSubsystemByName("plant")

# Add trajectory systems
V_G_source = builder.AddSystem(TrajectorySource(traj_V_G))
controller = builder.AddSystem(PseudoInverseController(plant))
integrator = builder.AddSystem(Integrator(7))
wsg_source = builder.AddSystem(TrajectorySource(traj_wsg_command))

# Connect everything
builder.Connect(V_G_source.get_output_port(), controller.GetInputPort("V_WG"))
builder.Connect(controller.GetOutputPort("iiwa.velocity"), integrator.get_input_port())
builder.Connect(integrator.get_output_port(), station.GetInputPort("iiwa.position"))
builder.Connect(station.GetOutputPort("iiwa.position_measured"), controller.GetInputPort("iiwa.position"))
builder.Connect(wsg_source.get_output_port(), station.GetInputPort("wsg.position"))

visualize_everything(station, plant)
diagram = builder.Build()

# Create simulator and set initial conditions
simulator = Simulator(diagram)
context = simulator.get_mutable_context()
plant_context = plant.GetMyContextFromRoot(context)


################ TEMP
###############








# Set letter position in simulation context
letter_model, letter_body = get_model_body(plant)
plant.SetFreeBodyPose(
    plant_context,
    letter_body,
    RigidTransform(R_WL, p0_letter),
)

# Set integrator to match actual starting position
integrator.set_integral_value(
    integrator.GetMyContextFromRoot(context),
    plant.GetPositions(plant_context, plant.GetModelInstanceByName("iiwa")),
)

diagram.ForcedPublish(context)
print(f"Simulation will run for {traj_V_G.end_time()} seconds")

meshcat.StartRecording()
if running_as_notebook:
    simulator.set_target_realtime_rate(1.0)
simulator.AdvanceTo(5)
meshcat.StopRecording()
meshcat.PublishRecording()

Simulation will run for 3.519275109384609 seconds


In [12]:
# After creating simulator and getting context
simulator = Simulator(diagram)
context = simulator.get_mutable_context()
plant_context = plant.GetMyContextFromRoot(context)

# Get initial joint angles
q_initial = plant.GetPositions(plant_context, plant.GetModelInstanceByName("iiwa"))
print(f"Initial joint angles: {q_initial}")

# Compute forward kinematics: get end effector pose from joint angles
# (This is what EvalBodyPoseInWorld does - it computes FK from current joint positions)
X_WG_initial_from_FK = plant.EvalBodyPoseInWorld(plant_context, plant.GetBodyByName("body"))
print(f"\nInitial end effector pose (from FK):")
print(f"  Position: {X_WG_initial_from_FK.translation()}")
print(f"  Rotation matrix:\n{X_WG_initial_from_FK.rotation().matrix()}")

# Use this as the desired final pose
X_WG_final_desired = X_WG_initial_from_FK

# Create a simple trajectory: start at initial, end at initial (should stay in place)
# Or you can create: start at initial, go somewhere, then return to initial
keyframes_test = [
    ("X_WG_start", X_WG_initial_from_FK, opened, 0.0),
    ("X_WG_final", X_WG_final_desired, opened, 2.0),  # Should return to start
]

sample_times_test = [0.0, 2.0]
gripper_poses_test = [X_WG_initial_from_FK, X_WG_final_desired]
finger_states_test = np.asarray([[opened, opened]])

traj_V_G_test, traj_wsg_command_test = make_bot_trajectory(
    gripper_poses_test, finger_states_test, sample_times_test
)

print(f"\n=== TEST TRAJECTORY ===")
print(f"Start pose: {X_WG_initial_from_FK.translation()}")
print(f"End pose: {X_WG_final_desired.translation()}")
print(f"Should stay in place (or return to start)")
print("=" * 50)

# Now replace your trajectory with this test trajectory
# V_G_source = builder.AddSystem(TrajectorySource(traj_V_G_test))  # Use test trajectory
# wsg_source = builder.AddSystem(TrajectorySource(traj_wsg_command_test))

Initial joint angles: [ 0.   0.5  0.  -1.8  0.   1.2  1. ]

Initial end effector pose (from FK):
  Position: [-0.42963119 -0.5         0.2652511 ]
  Rotation matrix:
[[-7.88001131e-01  3.50783228e-01  5.05969707e-01]
 [-5.40302306e-01 -3.05208641e-16 -8.41470985e-01]
 [-2.95173908e-01 -9.36456687e-01  1.89528987e-01]]

=== TEST TRAJECTORY ===
Start pose: [-0.42963119 -0.5         0.2652511 ]
End pose: [-0.42963119 -0.5         0.2652511 ]
Should stay in place (or return to start)


RigidTransform(
  R=RotationMatrix([
    [1.7042439132883673e-16, 0.3507832276896201, 0.9364566872907966],
    [-1.0000000000000004, -3.567338779378652e-16, 3.802498029544813e-16],
    [3.0978077131128555e-16, -0.9364566872907975, 0.35078322768961995],
  ]),
  p=[-0.42963119461589844, -0.5000000000000003, 0.2652510977908333],
)
Letter will reach 1.5m in 2.02 seconds
Predicted catch location: x=0.000, y=0.000, z=1.500

GRASP GEOMETRY CHECK
Object position (world):  x= 0.000, y= 0.000, z= 1.500
Gripper position (world): x= 0.000, y= 0.100, z= 1.400
Vertical offset: -0.100m
✓ Gripper is below object

Gripper orientation:
  Z-axis (approach): [ 0.00, -0.00, -1.00] ✗ (should point UP!)
  Y-axis (fingers):  [ 0.00, -1.00,  0.00] ✓ (horizontal)

X_WGpick: 
RigidTransform(
  R=RotationMatrix([
    [1.0, 0.0, 0.0],
    [0.0, -1.0, -1.2246467991473532e-16],
    [0.0, 1.2246467991473532e-16, -1.0],
  ]),
  p=[0.0, 0.1, 1.4],
)
DONE
Trajectory times: [0.0, np.float64(0.5192751093846089), np.float6

Simulation will run for 3.519275109384609 seconds
